In [17]:
import iSQL

In [18]:
R = iSQL.parser("NFL.db")

In [19]:
import sqlite3

db = sqlite3.connect("NFL.db")
cursor = db.cursor()

cursor.execute("""SELECT P.name, R.jersey, R.position
                    FROM Player P, PlayedFor R, Team T
                    WHERE P.pid=R.pid AND R.tid=T.tid
                        AND T.mascot='chiefs' AND R.year=2019 AND R.jersey<>'--'
                    ORDER BY R.jersey""")
print(" Name Jersey Position")
for row in sorted(cursor, key=lambda tup: int(tup[1])):
    if (int(row[1]) < 20):
        print("%20s %5s %6s" % row)
        
db.close()

 Name Jersey Position
         Joe Fortson     1     WR
     Dustin Colquitt     2      P
          Chad Henne     4     QB
   De'Anthony Thomas     5     WR
            Jack Fox     6      P
     Harrison Butker     7      K
        Chase Litton     8     QB
        Kyle Shurmur     9     QB
         Tyreek Hill    10     WR
   Demarcus Robinson    11     WR
       Gehrig Dieter    12     WR
       Byron Pringle    13     WR
       Sammy Watkins    14     WR
     Patrick Mahomes    15     QB
      Mecole Hardman    17     WR


In [20]:
import sqlite3
import datetime

db = sqlite3.connect("NFL.db")
cursor = db.cursor()

cursor.execute("""SELECT P.name, R.jersey, P.dob, MIN(R.year), T.mascot
                    FROM Player P, PlayedFor R, Team T
                    WHERE P.pid=R.pid AND R.tid=T.tid AND dob<>'--'
                        AND P.pid in (SELECT pid FROM PlayedFor
                    WHERE year=2019 AND position='QB')
                    GROUP BY P.pid
                    ORDER BY P.dob""")
print(" Name Jersey Age Seasons Team")

for row in cursor:
    ymd = [int(v) for v in row[2].split('-')]
    age = int((datetime.date.today() - datetime.date(ymd[0],ymd[1],ymd[2])).days/365.25)
    seasons = datetime.date.today().year - int(row[3])
    print("%20s %5s %6d %6d %18s" % (row[0],row[1],age,seasons,row[4]))

db.close()

 Name Jersey Age Seasons Team
          Mike White     3     62      1            cowboys
           Tom Brady    12     42     19           patriots
          Drew Brees     9     40     18           chargers
         Josh McCown    12     40     17          cardinals
         Eli Manning    10     38     15             giants
         Matt Schaub     8     38     15            falcons
       Philip Rivers    17     37     15           chargers
  Ben Roethlisberger     7     37     15           steelers
    Ryan Fitzpatrick    12     36     14               rams
       Aaron Rodgers    12     35     14            packers
          Alex Smith    11     35     14              49ers
        Drew Stanton     5     35     11              lions
          Matt Moore     3     35     12           panthers
          Joe Flacco     5     34     11             ravens
           Matt Ryan     2     34     11            falcons
          Chad Henne     7     34     11           dolphins
         B

In [32]:
import time
import sqlite3

Q2 = """SELECT P.name, COUNT(*) as cnt
 FROM Player P, PlayedFor R, Game G
 WHERE R.pid=P.pid AND G.season=R.year
 AND ((R.tid=G.vtid AND G.vscore>G.hscore) OR (R.tid=G.htid AND G.hscore>G.vscore))
 GROUP BY R.pid
 HAVING COUNT(*) > 200"""

db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

cursor.execute("CREATE INDEX IF NOT EXISTS SeasonVis ON Game(season,vtid)")
start = time.time()
cursor.execute(Q2)
playerGames = []
for row in cursor:
    playerGames.append((row['cnt'],row['name'],row['position']))
print("losing players = %d (%6.4f secs)" % (len(playerGames), time.time()-start))
for row in sorted(playerGames, reverse=True):
 print(row)

IndexError: No item with that key